## Setup

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop
from kepler.pandas.decorators import RingerDecorator
from itertools import product
import rootplotlib as rpl
import mplhep as hep
import root_numpy
import ROOT
ROOT.gStyle.SetOptStat(0);
import numpy as np
import pandas as pd
import glob
import os
import logging
import matplotlib.pyplot as plt
plt.style.use(hep.style.ROOT)
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
import joblib

from packages.generators import ring_percentages, RingGenerator
from packages.plotting import make_plot_fig, root2fig, COLORS, MARKERS, root2fig
from packages.utils import get_logger
from packages.constants import DROP_COLS, L1SEEDS_PER_ENERGY, CRITERIA_CONF_NAMES, ENERGY_CHAINS, TRIG_STEPS, HOME_PATH

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence


## Parameters

In [2]:
datasetpath = os.path.join(HOME_PATH, 
   'data/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.25bins.v2')

modelpaths = [
    'tunings/TrigL2_20180125_v8',
    'tunings/TrigL2_20210907_v12 (v8.1-2)',
    'tunings/TrigL2_20220704_v20',
    'tunings/TrigL2_20220802_v20.1',
    'tunings/TrigL2_20210306_v10',
    'tunings/TrigL2_20220822_v10.1'
]

output_dir = 'analysis/chain_simulation_test'
cutbased = True
plot_vars = ['et', 'eta', 'pt', 'mu']
values = ['pd']

chain_names = [
    'e24_lhtight_nod0_{strategy}_ivarloose',
    'e26_lhtight_nod0_{strategy}_ivarloose',
    'e60_lhmedium_nod0_{strategy}_L1EM24VHI',
    'e140_lhloose_nod0_{strategy}'
]

trigger_steps = ['L2Calo', 'L2', 'EFCalo', 'HLT']
dev = False
log = False

colors = [1, 601, 417, 633, 616, 401, 922]
markers = [33, 22, 23, 30, 20, 34, 21]

# Initializing logger
analysis_logger = get_logger('run_analysis', file=log)

## Loading chains

In [3]:
analysis_logger.info('Loading chains')
trigger_strategies = ['noringer'] if cutbased else list()
aux_conf_name = CRITERIA_CONF_NAMES['tight']
for modelpath in modelpaths:
    confpath = os.path.join(modelpath, aux_conf_name)
    env = ROOT.TEnv(confpath)
    ringer_version = env.GetValue("__version__", '')
    ringer_name = f'ringer_{ringer_version}'
    trigger_strategies.append(ringer_name)

2022-08-30 11:43:34,111 - Loading chains


In [4]:
trigger_strategies

['noringer',
 'ringer_v8',
 'ringer_v12',
 'ringer_v20',
 'ringer_v20.1',
 'ringer_v10',
 'ringer_v10.1']

In [5]:
strat_chains = dict()
for trig_strat in trigger_strategies:
    parquet_file = trig_strat + '.parquet'
    chainpath = os.path.join(datasetpath, 'simulated_chains', parquet_file)
    if dev:
        chainpath = os.path.join(chainpath, f'{trig_strat}_et4_eta4.parquet')
    analysis_logger.info(f'Loading: {parquet_file}')
    strat_chains[trig_strat] = pd.read_parquet(chainpath)

2022-08-30 11:43:34,151 - Loading: noringer.parquet
2022-08-30 11:43:34,220 - Loading: ringer_v8.parquet
2022-08-30 11:43:34,232 - Loading: ringer_v12.parquet
2022-08-30 11:43:34,242 - Loading: ringer_v20.parquet
2022-08-30 11:43:34,251 - Loading: ringer_v20.1.parquet
2022-08-30 11:43:34,262 - Loading: ringer_v10.parquet
2022-08-30 11:43:34,271 - Loading: ringer_v10.1.parquet


## Plotting

In [6]:
analysis_logger.info('Making plots')
for value, var, chain_name, step in product(values, plot_vars, chain_names, trigger_steps):
    plot_dir = os.path.join(output_dir, value, var)
    analysis_logger.info(f'Plotting value: {value}, step: {step}, chain_name: {chain_name}, var: {var}')
    make_plot_fig(strat_chains, step, chain_name, trigger_strategies, 
                  plot_dir , var, value, True, markers, colors)

2022-08-30 11:43:34,287 - Making plots
2022-08-30 11:43:34,289 - Plotting value: pd, step: L2Calo, chain_name: e24_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-30 11:43:34,502 - Plotting value: pd, step: L2, chain_name: e24_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-30 11:43:34,561 - Plotting value: pd, step: EFCalo, chain_name: e24_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-30 11:43:34,622 - Plotting value: pd, step: HLT, chain_name: e24_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-30 11:43:34,682 - Plotting value: pd, step: L2Calo, chain_name: e26_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-30 11:43:34,751 - Plotting value: pd, step: L2, chain_name: e26_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-30 11:43:34,810 - Plotting value: pd, step: EFCalo, chain_name: e26_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-30 11:43:34,871 - Plotting value: pd, step: HLT, chain_name: e26_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-30 11:43:34,931

## Regenerate from cached data

In [12]:
def cached_root2fig(root_info_dir, plot_name, colors, markers, save=False):

    var, value, step, energy, criterion = plot_name.split('_')
    plotpath = os.path.join(root_info_dir, plot_name)
    root_plots = joblib.load(plotpath + '.joblib')[1:]
    labels = joblib.load(plotpath + '_labels.joblib')[1:]
    n_strats = len(root_plots)
    colors = COLORS[:n_strats] if colors is None else colors
    markers = MARKERS[:n_strats] if markers is None else markers

    _, fig = root2fig(root_plots, labels, colors, markers, plot_name)

    if save:
        plotpath = os.path.join(root_info_dir.replace('2022_09_01_v20.1_all_data17', 'refact'), plot_name)
        #plotpath = os.path.join(root_info_dir, plot_name)
        analysis_logger.info(plotpath)
        fig.savefig(plotpath + '.pdf')
        fig.savefig(plotpath + '.png')

    return plot_name, fig, labels

In [10]:
analysis_logger = get_logger('cache_analysis', file=False)
colors = [1, 601, 417, 633, 616, 401, 922][1:]
markers = [33, 22, 23, 30, 20, 34, 21][1:]
save=True

In [18]:
plot_base_dir = os.path.join(HOME_PATH, 'workspace', 'ringer', 'analysis', '2022_09_12_v12.1_data17')
for plot_label in glob.iglob(os.path.join(plot_base_dir, 'fr', '*', '*_labels.joblib')):
    plot_dir, plot_name = os.path.split(plot_label)
    plot_name = plot_name.replace('_labels.joblib', '')
    analysis_logger.info(f'Processing {plot_name}')
    root_plots = joblib.load(plot_label.replace('_labels', ''))[1:]
    labels = joblib.load(plot_label)[1:]
    n_strats = len(root_plots)
    colors = COLORS[:n_strats] if colors is None else colors
    markers = MARKERS[:n_strats] if markers is None else markers

    _, fig = root2fig(root_plots, labels, colors, markers, plot_name)
    # output_dir = plot_dir + '_refact'
    # if not os.path.exists(output_dir):
    #     os.makedirs(output_dir)
    # analysis_logger.info(f'Saving {os.path.join(output_dir, plot_name)}')

    if save:
        fig.savefig(os.path.join(plot_dir, plot_name + '.pdf'))
        fig.savefig(os.path.join(plot_dir, plot_name + '.png'))

2022-09-14 11:31:00,468 - Processing et_fr_L2Calo_e24_lhtight
2022-09-14 11:31:00,468 - Processing et_fr_L2Calo_e24_lhtight
2022-09-14 11:31:00,468 - Processing et_fr_L2Calo_e24_lhtight
2022-09-14 11:31:00,530 - Processing et_fr_L2_e24_lhtight
2022-09-14 11:31:00,530 - Processing et_fr_L2_e24_lhtight
2022-09-14 11:31:00,530 - Processing et_fr_L2_e24_lhtight
2022-09-14 11:31:00,572 - Processing et_fr_EFCalo_e24_lhtight
2022-09-14 11:31:00,572 - Processing et_fr_EFCalo_e24_lhtight
2022-09-14 11:31:00,572 - Processing et_fr_EFCalo_e24_lhtight
2022-09-14 11:31:00,616 - Processing et_fr_HLT_e24_lhtight
2022-09-14 11:31:00,616 - Processing et_fr_HLT_e24_lhtight
2022-09-14 11:31:00,616 - Processing et_fr_HLT_e24_lhtight
2022-09-14 11:31:00,654 - Processing et_fr_L2Calo_e26_lhtight
2022-09-14 11:31:00,654 - Processing et_fr_L2Calo_e26_lhtight
2022-09-14 11:31:00,654 - Processing et_fr_L2Calo_e26_lhtight
2022-09-14 11:31:00,697 - Processing et_fr_L2_e26_lhtight
2022-09-14 11:31:00,697 - Process